<a href="https://github.com/gilbutITbook/006975">코드</a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


# 텍스트와 시퀀스를 위한 딥러닝
## 시퀀스 데이터를 처리하는 기본적인 모델
- 순환 신경망
- 1D Convnet

## 자연어 처리 과정
- 자연어 -> 토큰화 -> 벡터화

# 6.1 텍스트 데이터 다루기

### 단어수준의 one-hot 인코딩

In [ ]:
# 6-1 단어 수준의 원-핫 인코딩하기(간단한 예)
import numpy as np

# 초기 데이터: 각 원소가 셈플
# 여기서는 하나의 샘플이 하나의 문장이지만, 문서 전체가 될 수도 있다.

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 데이터 내의 모든 토큰의 인덱스를 구한다
token_index = {}
for sample in samples:
  # split으로 토큰으로 나눔
  # 실전에서는 구둣점, 특문 사용
  for word in sample.split():
    if word not in token_index:
      token_index[word] = len(token_index) + 1
      # 인덱스 0은 사용하지 않는다

# 샘플을 벡터로 변환합니다
# 각 샘플에서 max_length까지 단어만 사용
max_length = 10

# 결과를 저장할 배열
results = np.zeros((len(samples),max_length,max(token_index.values())+1))
for i, sample in enumerate(samples):
  for j, character in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(character)
    results[i, j, index] = 1.

### 문자 수준 원-핫 인코딩

In [ ]:
# 6-2 문자 수준 원-핫 인코딩하기(간단한 예)
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable
token_index = dict(zip(characters,range(1,len(characters)+1)))

max_length = 50
results = np.zeros((len(samples),max_length,max(token_index.values())+1))
for i, sample in enumerate(samples):
  for j, character in enumerate(sample[:max_length]):
    index = token_index.get(character)
    results[i,j,index] = 1.

In [ ]:
results[1][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### 케라스를 사용한 단어 수준의 원-핫 인코딩하기

In [ ]:
# 케라스를 사용한 단어 수준의 원-핫 인코딩하기
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 가장 빈도가 높은 1,000개의 단어만 선택하도록 Tokenizer 객체를 만듭니다.
tokenizer = Tokenizer(num_words=1000)
# 단어 인덱스를 구축합니다.
tokenizer.fit_on_texts(samples)

# 문자열을 정수 인덱스의 리스트로 변환합니다.
sequences = tokenizer.texts_to_sequences(samples)

# 직접 원-핫 이진 벡터 표현을 얻을 수 있습니다.
# 원-핫 인코딩 외에 다른 벡터화 방법들도 제공합니다!
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 계산된 단어 인덱스를 구합니다.
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


### 해싱을 이용한 단어수준의 원-핫 인코딩

In [ ]:
# 6-4 해싱 기법을 사용한 단어 수준의 원-핫 인코딩하기(간단한 예)
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 단어를 크기가 1,000인 벡터로 저장합니다.
# 1,000개(또는 그이상)의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될 것입니다
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # 단어를 해싱하여 0과 1,000 사이의 랜덤한 정수 인덱스로 변환합니다.
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

### 원핫 인코딩의 특징
1. sparse하다
2. 고차원이다
3. 수동 인코딩이다

### 단어 임베딩(밀집 단어 벡터) 방법
1. 관심 대상인 무제와 함께 단어 임베딩을 학습
2. 사전 훈련된 단어 임베딩을 로드

### 직접 임베딩 훈련

In [ ]:
# 6-5 Embedding 층의 객체 생성하기
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64) #가능한 토큰의 개수 == max(단어index)+ 1, 임베딩차원

In [ ]:
# 6-6 Embdding 층에 사용할 IMDB 데이터 고르기
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000   # 특성으로 사용할 단어의 수
maxlen = 20            # 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용)

(x_train, y_train), (x_test,y_test) = imdb.load_data(num_words=max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17465344/17464789 [==============================] - 2s 0us/step


In [ ]:
# 6-7 IMDB데이터에 Embedding 층과 분류기 사용하기
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model=Sequential()
model.add(Embedding(10000,8,input_length=maxlen))     # 나중에 Flatten으로 펼치기 위해 input_length를 지정해줌
                                                      # 이 층의 출력은 (samples, maxlen, 8)

model.add(Flatten()) # 3D임베딩 텐서를 (samples, maxlen*8)의 크기의 2D 텐서로 펼친다

model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()

history= model.fit(x_train,y_train,
                   epochs=10,
                   batch_size=32,
                   validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 1s 2ms/step - loss: 0.6601 - acc: 0.6449 - val_loss: 0.6023 - val_acc: 0.7102
Epoch 2/10
625/625 [==============================] - 1s 1ms/step - loss: 0.5298 - acc: 0.7551 - val_loss: 0.5210 - val_acc: 0.7356
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.4578 - acc: 0.7895 - val_loss: 0.4998 - va

### 사전 훈련된 임베딩

# 6.2 순환 신경망 이해하기

# 6.3 순환 신경망의 고급 사용법

# 6.4 컨브넷을 사용한 시퀀스 처리